In [ ]:


    



    

    # 4. Read in test data and apply data prep from step 1
    test_df = data_prep.load_data(test_filepath)
    champ_categorical_encoders = data_prep.CategoricalEncoders(champ_cat_enc, champ_ord_enc)
    test_data_prep_inputs = data_prep.TestDataPrepInputs(test_df, champ_categorical_encoders, include_categoricals)
    test_X = test_data_prep_inputs.test_data_prep()

    # 5. predict test data and output to csv
    # the model was trained against log transformed target. Invert log for predictions
    test_y = np.exp(champ_model.predict(test_X))
    print(test_X.head())

    test_y = pd.DataFrame(test_y, columns=[target_variable], index=test_X.index)

    test_y.to_csv('prediction.csv')

In [1]:
import lightgbm
import numpy as np
import pandas as pd
from sklearn import ensemble, linear_model # type: ignore
import xgboost

import cross_validation
import data_prep

In [2]:
train_filepath = "data/train.csv"
test_filepath = "data/test.csv"

target_variable = 'SalePrice'
# temporary variable to remove categoricals
include_categoricals = True
# number of folds to cross-validate across
folds = 2
# model to cross validate
lr = linear_model.LinearRegression()
ohe_bool = False
n_iter = 2

In [3]:
lgbm = lightgbm.LGBMRegressor(verbose=-1)
lgbm_param_grid = {
    'max_depth': tuple(range(1, 100)),
    'num_leaves': tuple(range(1, 100000, 100))
}

In [4]:
# 1. read in training data, perform data prep
train_df = data_prep.load_data(train_filepath)
train_data_prep_inputs = data_prep.TrainDataPrepInputs(train_df, ohe_bool, target_variable, include_categoricals)
train_data_prep_outputs = train_data_prep_inputs.train_data_prep()

In [5]:
lgbm = cross_validation.bayes_cross_validation(lgbm, train_data_prep_outputs.train_data, lgbm_param_grid, n_iter)
lgbm_param_scores = cross_validation.save_model_performance_parameters(lgbm, folds, train_data_prep_outputs)

In [6]:
# 3. Compare performance and select best model
# models = [lr_param_scores, rf_param_scores, xgb_param_scores, lgbm_param_scores, en_param_scores, lasso_param_scores,
#           ridge_param_scores, br_param_scores]
models = [lgbm_param_scores]
param_scores = pd.DataFrame(models).sort_values('score', ascending=False)

In [7]:
print(param_scores[['model', 'score', 'standard_dev']])

# take only first row in case of ties (I don't care which model if they're tied)
champ = param_scores[param_scores.score == param_scores.score.max()].reset_index()
champ_model = champ.model[0]
champ_ord_enc = champ.ordinal_encoder[0]
champ_cat_enc = champ.categorical_encoder[0]

                                               model    score  standard_dev
0  BayesSearchCV(cv=4, estimator=LGBMRegressor(ve...  0.88335      0.006828


In [8]:
train_data_prep_outputs.train_data.X.BsmtCond

Id
1       4.0
2       4.0
3       4.0
4       1.0
5       4.0
       ... 
1456    4.0
1457    4.0
1458    1.0
1459    4.0
1460    4.0
Name: BsmtCond, Length: 1460, dtype: float64

In [9]:
# 4. Read in test data and apply data prep from step 1
test_df = data_prep.load_data(test_filepath)
champ_categorical_encoders = data_prep.CategoricalEncoders(champ_cat_enc, champ_ord_enc)
test_data_prep_inputs = data_prep.TestDataPrepInputs(test_df, champ_categorical_encoders, include_categoricals)


In [10]:
test_data_prep_inputs.test_df.Alley[test_data_prep_inputs.test_df.Alley.isnull() == False]

36      Pave
37      Pave
38      Pave
39      Pave
42      Pave
        ... 
1330    Grvl
1335    Pave
1337    Grvl
1410    Grvl
1411    Grvl
Name: Alley, Length: 107, dtype: object

In [11]:
test_X = test_data_prep_inputs.test_data_prep()

Id                 object
MSSubClass       category
MSZoning         category
LotFrontage       float64
LotArea             int64
                   ...   
MoSold              int64
YrSold              int64
SaleType         category
SaleCondition    category
TotalBath         float64
Length: 81, dtype: object
MSSubClass       category
MSZoning         category
LotFrontage       float64
LotArea           float64
Street           category
                   ...   
MoSold            float64
YrSold            float64
SaleType         category
SaleCondition    category
TotalBath         float64
Length: 80, dtype: object


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- BsmtCond
- BsmtExposure
- BsmtFinType1
- BsmtFinType2
- BsmtQual
- ...
Feature names seen at fit time, yet now missing:
- Alley
- BldgType
- Condition1
- Condition2
- Electrical
- ...
